In [1]:
# pip3 install opencv-python numpy matplotlib
import cv2 as cv
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os


In [2]:
# Source: https://medium.com/analytics-vidhya/binarization-of-image-using-numpy-65df2b82e189
def read_this(image_file, gray_scale=False):
    image_src = cv.imread(image_file)
    if gray_scale:
        image_src = cv.cvtColor(image_src, cv.COLOR_BGR2GRAY)
    else:
        image_src = cv.cvtColor(image_src, cv.COLOR_BGR2RGB)
    return image_src

In [3]:
def ApplyOtsu(image_file, thresh_val=127, gray_scale=True):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    blur = cv.GaussianBlur(image_src,(5,5),0)
    th, image_b = cv.threshold(image_src, 0, 255, cv.THRESH_BINARY+ cv.THRESH_OTSU)
    return Image.fromarray(image_b)

In [4]:
def save_image(image, old_name, dir):
    if('_binarized' not in old_name):
        index = dir.find('.jpg')
        final_string = dir[:index] + '_binarized' + dir[index:]
        image.save(final_string)
    

def main():
    dir = os.getcwd()
    try:
        #assuming this file is one directory upper than images
        directory = os.fsencode(dir + '/images/')
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".jpg"):
                image_dir = dir + '/images/' + filename
                binarized_image = ApplyOtsu(image_file=image_dir)
                save_image(binarized_image, filename, image_dir)
            else:
                continue
                
    except Exception as e:
        print(e)

if __name__ == "__main__":
    main()  

In [26]:
import cv2
import numpy as np
#identify paragraps
# Load image, grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('./images/fmb-cb-55-075r_binarized.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Create rectangular structuring element and dilate
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.dilate(thresh, kernel, iterations=4)

# Find contours and draw rectangle
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

cropped = [];
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    cropped.append(image[y:y+h, x:x+w])
    cv2.rectangle(image, (x, y), (x + w, y + h), (30,200,10), 2)

#cv2.imshow('thresh', thresh)
#cv2.imshow('dilate', dilate)
cv2.imshow('image', image)
i = 0
for k in cropped:
    cv2.imwrite('cropped_image' + str(i) +'.png', k)
    i= i+1
cv2.waitKey()

-1